In [1]:
!nvidia-smi

Sun May 14 12:48:50 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro RTX 6000                 On | 00000000:1B:00.0 Off |                  Off |
| 33%   26C    P8               10W / 260W|  23893MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from time import time
import tensorflow as tf 
import matplotlib.pyplot as plt
from Loader import train_data, test_data
from matplotlib import colors

2023-05-14 12:48:52.037454: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.operation_timeout_in_ms=60000
sess = tf.compat.v1.Session(config=config)

2023-05-14 12:48:53.470810: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 12:48:53.477756: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-05-14 12:48:53.478803: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-05-14 12:48:55.221418: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:1b:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.64GiB deviceMemoryBandwidth: 625.94GiB/s
2023-05-14 12:48:55.221966: I tensorflow/core/common_runtime/gpu/gpu_devic

In [4]:
####################################################################
# Following code releases unused GPU memory for others to use.
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)
####################################################################

2023-05-14 12:48:59.158269: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-05-14 12:48:59.158917: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:1b:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.64GiB deviceMemoryBandwidth: 625.94GiB/s
2023-05-14 12:48:59.159357: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:1c:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.64GiB deviceMemoryBandwidth: 625.94GiB/s
2023-05-14 12:48:59.159770: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:1d:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.64GiB deviceMemoryBandwidth: 625.94GiB/s
2023-05-14 12:48:59.1

In [5]:
df_train = train_data()
print(df_train.shape)
df_train.head(3)

(100000, 3073)


,0,1,2,3,4,5,6,7,8,9,...,3063,3064,3065,3066,3067,3068,3069,3070,3071,label
0,114,112,113,119,117,118,117,117,115,111,...,100,91,86,99,91,88,98,90,87,Fake
1,206,204,205,215,213,214,224,222,223,228,...,126,125,123,124,123,121,123,122,120,Fake
2,153,185,208,130,162,185,153,185,208,162,...,167,178,172,166,177,171,165,176,170,Fake


In [6]:
df_sample = df_train.sample(frac=0.1, random_state=0)
print(df_sample.shape)
X = df_sample.drop('label', axis=1)
X = (X-X.mean())/X.std()
y = df_sample.label

(10000, 3073)


In [7]:
# reg_stren = [0.0005, 0.001, 0.005]
reg_stren = np.linspace(0.0001, 0.001, 5)
print(reg_stren)
pen = ['l2', 'l1']
solv = ['saga']
iter = [5000, 10000]
grid = {'C':reg_stren, 'penalty':pen, 'solver':solv, 'max_iter':iter}

[0.0001   0.000325 0.00055  0.000775 0.001   ]


In [ ]:
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)
with strategy.scope():

    
    ####################################################################
    # Baseline Logistic Regression
    time_start = time()
    lgr = LogisticRegression(n_jobs = -1)
    lgrCV = GridSearchCV(lgr, param_grid=grid, return_train_score=True, n_jobs=-1)
    # lgr.fit(X,y)
    lgrCV.fit(X,y)

    time_stop = time()
    print('Comp Time:', (time_stop-time_start)/60, 'min')
    
#     ####################################################################
#     # Cross Validation
    
#     ## L2 Regularization
#     ts = time()
#     lgrcv2 = LogisticRegressionCV(cv=3, 
#                                   random_state=0, 
#                                   n_jobs=-1,
#                                   penalty='l2',
#                                   solver='sag',
#                                   max_iter = 5000)

#     lgrcv2.fit(X,y)
#     tss= time()
#     print('CV l2 Time:', (tss-ts)/60, 'min')
#     print()
#     print('CV l2 Baseline:', lgrcv2.score(X,y))
#     print()
    
    
#     ####################################################################
    
#     ## L1 Regularization 
#     ts = time()
#     lgrcv1 = LogisticRegressionCV(cv=3, 
#                                   random_state=0, 
#                                   n_jobs=-1,
#                                   penalty='l1',
#                                   solver='saga',
#                                   max_iter = 10000)

#     lgrcv1.fit(X,y)
#     tss= time()
#     print('CV l1 Time:', (tss-ts)/60, 'min')
#     print()
#     print('CV l1 Baseline:', lgrcv1.score(X,y))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


2023-05-14 12:49:32.002693: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-05-14 12:49:32.003523: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:1b:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.64GiB deviceMemoryBandwidth: 625.94GiB/s
2023-05-14 12:49:32.004086: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:1c:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.64GiB deviceMemoryBandwidth: 625.94GiB/s
2023-05-14 12:49:32.004601: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:1d:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.64GiB deviceMemoryBandwidth: 625.94GiB/s
2023-05-14 12:49:32.0

In [ ]:
print('Hyper Parameters:', lgrCV.best_params_)
print('R2', lgrCV.best_score_)

In [ ]:
lgr = LogisticRegression(C=lgrCV.best_params_['C'],
                         penalty=lgrCV.best_params_['penalty'],
                         solver=lgrCV.best_params_['solver'],
                         max_iter=lgrCV.best_params_['max_iter'])
lgr.fit(X, y)

In [ ]:
ix = lgrCV.cv_results_['mean_test_score'].argmax()

In [ ]:
accuracy_train = lgrCV.cv_results_['mean_train_score'][ix].round(4)
accuracy_valid = lgrCV.cv_results_['mean_test_score'].max().round(4)

In [ ]:
print('Training Accuracy:', accuracy_train)
print()
print('Validation Accuracy:', accuracy_valid)

In [ ]:
df_test = test_data()
print(df_test.shape)
df_test.head(3)

In [ ]:
# bestL2_C = lgrcv2.C_
# bestL1_C = lgrcv1.C_

# print('Best L2 param:', bestL2_C)
# print()
# print('Best L1 param:', bestL1_C)

In [ ]:
# ts = time()
# lgrcv2 = LogisticRegressionCV(cv=5,
#                               Cs=[bestL2_C],
#                               random_state=0, 
#                               n_jobs=-1,
#                               penalty='l2',
#                               solver='sag',
#                               max_iter = 5000)

# lgrcv2.fit(X,y)
# tss = time()
# print('CV l2 Time:', tss-ts)
# print()
# print('CV l2 Baseline:', lgrcv2.score(X,y))

In [ ]:
# tf.debugging.set_log_device_placement(True)
# gpus = tf.config.list_logical_devices('GPU')
# strategy = tf.distribute.MirroredStrategy(gpus)
# with strategy.scope():
#     ts = time()
#     lgrcv1 = LogisticRegressionCV(cv=5,
#                                   Cs=[bestL1_C],
#                                   random_state=0, 
#                                   n_jobs=-1,
#                                   penalty='l1',
#                                   solver='saga',
#                                   max_iter = 10000)

#     lgrcv1.fit(X,y)
#     tss = time()
#     print('CV l1 Time:', tss-ts)
#     print()
#     print('CV l1 Baseline:', lgrcv1.score(X,y))

In [ ]:
# resultsl2 = lgrcv2.coef_
# resultsl2 = resultsl2.reshape([32,32,3])
# x = []
# y = []

# xred = []
# yred = []

# xgreen = []
# ygreen = []

# xblue = []
# yblue = []

# weight = [0.001, 0.005, 0.01]

# cmap = plt.cm.get_cmap('jet')

# for w in range(len(weight)):
#     for rgb in range(3):
#         mask = np.abs(resultsl2[:, :, rgb]) > weight[w]
#         heat_map = np.zeros((32, 32))
#         heat_map[mask] = np.abs(resultsl2[mask][:, rgb])

#         plt.imshow(heat_map, cmap=cmap, alpha=0.5)  # Display the heat map
#         plt.colorbar()

#         plt.show()